In [1]:
from PIL import Image
import os

In [3]:
import os
import rasterio
from rasterio.enums import Resampling
from tqdm import tqdm

def upscale_image_test(folder):
    directory = f"../../data/test/{folder}"
    output_directory = f"../../data/test_downscaled/{folder}"
    os.makedirs(output_directory, exist_ok=True)

    # Loop through each file in the directory
    for filename in tqdm(os.listdir(directory), desc="Downscaling images"):
        if filename.lower().endswith('.tif') or filename.lower().endswith('.tiff'):
            file_path = os.path.join(directory, filename)
            # Open the image with rasterio
            with rasterio.open(file_path) as src:
                # Calculate new dimensions (50% scale)
                new_width = 128
                new_height = 128

                # Update the transformation matrix for the affine transformation of the image
                transform = src.transform * src.transform.scale(
                    (src.width / new_width),
                    (src.height / new_height)
                )

                # Set the metadata for the output image
                profile = src.profile
                profile.update({
                    'height': new_height,
                    'width': new_width,
                    'transform': transform
                })

                output_path = os.path.join(output_directory, filename)
                # Write the downscaled image
                with rasterio.open(output_path, 'w', **profile) as dst:
                    # Read the image data and resample
                    for i in range(1, src.count + 1):
                        data = src.read(
                            i,
                            out_shape=(new_height, new_width),
                            resampling=Resampling.bilinear
                        )
                        dst.write(data, i)

    print(f"Images have been upscaled and saved in {output_directory}")


In [4]:
upscale_image_test("catalonia")

Downscaling images: 100%|██████████| 2000/2000 [00:19<00:00, 104.64it/s]

Images have been downscaled and saved in ../../data/test_downscaled/catalonia


In [5]:
upscale_image_test("ar")

Downscaling images: 100%|██████████| 2000/2000 [00:19<00:00, 100.97it/s]

Images have been downscaled and saved in ../../data/test_downscaled/ar
